In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

from publicdata.census.files.metafiles import TableLookup
import shelve 


%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()

# Setup importing form this package
pkg.set_sys_path()
import pylib # Import the code for the package
from pylib.census_path_processing import * 

pkg

# Metadata for the 2020 ACS
`civicknowledge.com-census_meta-2020e5-1.1.1` Last Update: 2023-01-06T17:07:19

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [paths](notebooks/build.ipynb#paths_df)**. Column paths, processed by GPT
* ** [metadata](notebooks/build.ipynb#metadata_df)**.

In [3]:

year = 2020
release = 5

tl = TableLookup(year, release)

tdf = tl.tables_df
cdf = tl.columns_df

In [4]:
#
# Clean the pathnames
# 


consecutive_err = 0
while True:
    consecutive_err +=1
    try:
        run_path_cleaning()
        consecutive_err = 0
    except StopIteration:
        print("Done")
        break
    except Exception as e:
        print(e)

    if consecutive_err > 3:
        print("Too many errors")
        break

        
# Turn the database into a dataset, so we can save it in the package, for caching in future builds. 
# This is not currently loaded yet. 
with shelve.open('census_paths') as db:
    paths_df = pd.DataFrame(db.values())
    
paths_df.head()
        

name 'shelve' is not defined
name 'shelve' is not defined
name 'shelve' is not defined
name 'shelve' is not defined
Too many errors


,unique_id,path,name,description
0,B05007,/latin america/central america/mexico/entered ...,mexico_2010_later,People from Mexico who entered the U.S. after ...
1,B25129,/renter occupied/moved in 1990 to 1999,renter_moved_1990_1999,Renter occupied who moved in 1990 to 1999
2,B27011,/in labor force/employed/with health insurance...,employed_with_health_insurance,People who are employed and have health insura...
3,B26103F,/group quarters population/institutionalized g...,institutionalized_group_quarters_population,People who are living in institutionalized gro...
4,B26208,/group quarters population/noninstitutionalize...,group_quarters_population_noninstitutionalized...,People who live in noninstitutionalized group ...


In [5]:

with shelve.open('census_paths') as db:
    pdf = pd.DataFrame(db.values()).rename(
            columns={
                "unique_id": "table_id",
                "path": "filtered_path",
                "name": "path_name",
            }
        )


    mdf = add_rest_str(tdf, cdf)
    mdf = mdf.merge(pdf, on=["table_id", "filtered_path"], how="left").copy()

    mdf["rest_description"] = mdf.apply(make_restricted_description, axis=1)

    mdf["col_desc"] = mdf.stripped_title + " for " + mdf.rest_description

    metadata_df = mdf

metadata_df.head()

,table_id,seq,start_pos,title,universe,subject,race,table_age_range,table_age,table_sex,...,n_levels,level_1,level_2,level_3,level_4,restriction_str,path_name,description,rest_description,col_desc
0,B01001,[1],7,Sex By Age,Total population,Age-Sex,all,None,all,all,...,0,None,None,None,None,adults,NaN,NaN,adults,Population for adults
1,B01001,[1],7,Sex By Age,Total population,Age-Sex,all,None,all,all,...,0,None,None,None,None,men,NaN,NaN,men,Population for men
2,B01001,[1],7,Sex By Age,Total population,Age-Sex,all,None,all,all,...,0,None,None,None,None,"male children, infants and todlers",NaN,NaN,"male children, infants and todlers","Population for male children, infants and todlers"
3,B01001,[1],7,Sex By Age,Total population,Age-Sex,all,None,all,all,...,0,None,None,None,None,"boys, ages 5 to 9",NaN,NaN,"boys, ages 5 to 9","Population for boys, ages 5 to 9"
4,B01001,[1],7,Sex By Age,Total population,Age-Sex,all,None,all,all,...,0,None,None,None,None,"males, ages 10 to 14",NaN,NaN,"males, ages 10 to 14","Population for males, ages 10 to 14"


In [7]:
metadata_df.head().T

,0,1,2,3,4
table_id,B01001,B01001,B01001,B01001,B01001
seq,[1],[1],[1],[1],[1]
start_pos,7,7,7,7,7
title,Sex By Age,Sex By Age,Sex By Age,Sex By Age,Sex By Age
universe,Total population,Total population,Total population,Total population,Total population
subject,Age-Sex,Age-Sex,Age-Sex,Age-Sex,Age-Sex
race,all,all,all,all,all
table_age_range,None,None,None,None,None
table_age,all,all,all,all,all
table_sex,all,all,all,all,all
